# Conditions and loop

2015_12_15

조강익

sky8671@gmail.com

> Import libraries

In [4]:
from IPython.display import IFrame
from theano import tensor as T
from theano.ifelse import ifelse
import theano, time, numpy

> *내용은 전부 Theano site documentation에서 가지고 왔습니다.*

# Conditions

- 조건
    - "조건이 맞으면 주세요"
    - 프로그래밍 할 때 가장 원하는 것 중 top 순위

<img src="http://www.functionx.com/flowcharts/if.gif" width=400>

<br><br><br><br><br><br><br><br><br><br><br><br>

<img src="http://deeplearning.net/software/theano/_static/theano_logo_allblue_200x46.png">

In [3]:
IFrame('http://deeplearning.net/software/theano/tutorial/conditions.html', width=700, height=200)

1. IfElse
    - 조건에 맞는 것만 실행
2. Switch
    - 모두 선택 후 조건에 맞는 것 고름

# 예제

In [5]:
a,b = T.scalars('a', 'b')
x,y = T.matrices('x', 'y')

- a랑 b는 scalar
- x랑 y는 matrix가 될 것 임

#### 스위치는 아래와 같이 만든다

In [6]:
z_switch = T.switch(T.lt(a, b), T.mean(x), T.mean(y))

- `T.lt` 는 a가 b보다 작으면 True

#### ifelse는 아래와 같이 만든다

In [8]:
z_lazy = ifelse(T.lt(a, b), T.mean(x), T.mean(y))

####  function 만들기

In [12]:
f_switch = theano.function([a, b, x, y], z_switch,
                           mode=theano.Mode(linker='vm'))
f_lazyifelse = theano.function([a, b, x, y], z_lazy,
                               mode=theano.Mode(linker='vm'))

- `a, b, x, y` 를 input으로 하고
- `z_lazy` 의 결과를 받는다

#### 실제로 돌려보기

In [13]:
val1 = 0.
val2 = 1.
big_mat1 = numpy.ones((10000, 1000))
big_mat2 = numpy.ones((10000, 1000))

n_times = 10

tic = time.clock()
for i in xrange(n_times):
    f_switch(val1, val2, big_mat1, big_mat2)
print 'time spent evaluating both values %f sec' % (time.clock() - tic)

tic = time.clock()
for i in xrange(n_times):
    f_lazyifelse(val1, val2, big_mat1, big_mat2)
print 'time spent evaluating one value %f sec' % (time.clock() - tic)


time spent evaluating both values 0.255915 sec
time spent evaluating one value 0.151861 sec


- 걸리는 시간이 다름
    - switch는 둘다 돌리기 때문에 더 오래 걸림

<br><br><br><br><br><br><br><br><br><br><br><br><br>

 <img src="http://deeplearning.net/software/theano/_static/theano_logo_allblue_200x46.png">
 #Loop

<img src="http://cdn2.hubspot.net/hub/53/file-23117129-png/blog/images/closed-loop-marketing.png">

- Loop
    - 반복해주세요
    - 프로그래밍 할 때 가장 원하는 것 중 top 순위

In [18]:
IFrame('http://deeplearning.net/software/theano/library/scan.html#lib-scan', width=700, height=200)

- scan을 쓰는 것의 장점
    - symbolic graph로 표현됨
    - GPU 사용을 효율적으로 해줌
    - 순차적으로 gradient 계산 가능??
    - python loop보다 빠름
    - 메모리 효율적 관리가능

- 아래와 비슷한 component를 가지고 있다

```
result = 1
for i in xrange(k):
  result = result * A
```

- python loop
    - initial value
    - accumulation 
    - unchanging variable

## =

- Theano scan
    - `outputs_info`
    - automatic
    - `non_sequences`

## 예제1

In [19]:
import theano
import theano.tensor as T

k = T.iscalar("k")
A = T.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=T.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.
final_result = result[-1]

# compiled function that returns A**k
power = theano.function(inputs=[A,k], outputs=final_result, updates=updates)

print power(range(10),2)
print power(range(10),4)

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]
[  0.00000000e+00   1.00000000e+00   1.60000000e+01   8.10000000e+01
   2.56000000e+02   6.25000000e+02   1.29600000e+03   2.40100000e+03
   4.09600000e+03   6.56100000e+03]


/Users/kcho/Dropbox/Knowledges/python/virtualEnv/lib/python2.7/site-packages/theano/gof/cmodule.py:293: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  rval = __import__(module_name, {}, {}, [module_name])


```
result, updates = theano.scan(
    fn=lambda prior_result, A: prior_result * A,    # function
    outputs_info=T.ones_like(A),                    # 처음 값
    non_sequences=A,                                # 변화하지 않는 값
    n_steps=k)                                      # number of loops
```

## 예제2

In [20]:
import numpy

coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")

max_coefficients_supported = 10000

# Generate the components of the polynomial
components, updates = theano.scan(fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),
                                  outputs_info=None,
                                  sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],
                                  non_sequences=x)
# Sum them up
polynomial = components.sum()

# Compile a function
calculate_polynomial = theano.function(inputs=[coefficients, x], outputs=polynomial)

# Test
test_coefficients = numpy.asarray([1, 0, 2], dtype=numpy.float32)
test_value = 3
print calculate_polynomial(test_coefficients, test_value)
print 1.0 * (3 ** 0) + 0.0 * (3 ** 1) + 2.0 * (3 ** 2)

19.0
19.0


```
components, updates = theano.scan(
    fn=lambda coefficient, power, free_variable: coefficient * (free_variable ** power),  
        # x가 파워로 들어감
        
    outputs_info=None,                                                                    
        # 초기값이 없고, 매번 업데이트 되지 않음
        
    sequences=[coefficients, theano.tensor.arange(max_coefficients_supported)],                                 
    non_sequences=x) 
    
    
# Sum them up
polynomial = components.sum()    # 위에서 나온 값을 더한다
```

## 예제3

In [21]:
import numpy as np
import theano
import theano.tensor as T

up_to = T.iscalar("up_to")

# define a named function, rather than using lambda
def accumulate_by_adding(arange_val, sum_to_date):
    return sum_to_date + arange_val
seq = T.arange(up_to)

# An unauthorized implicit downcast from the dtype of 'seq', to that of
# 'T.as_tensor_variable(0)' which is of dtype 'int8' by default would occur
# if this instruction were to be used instead of the next one:
# outputs_info = T.as_tensor_variable(0)

outputs_info = T.as_tensor_variable(np.asarray(0, seq.dtype))
scan_result, scan_updates = theano.scan(fn=accumulate_by_adding,
                                        outputs_info=outputs_info,
                                        sequences=seq)
triangular_sequence = theano.function(inputs=[up_to], outputs=scan_result)

# test
some_num = 15
print(triangular_sequence(some_num))
print([n * (n + 1) // 2 for n in xrange(some_num)])

[  0   1   3   6  10  15  21  28  36  45  55  66  78  91 105]
[0, 1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105]


## 예제4

In [22]:
location = T.imatrix("location")
values = T.vector("values")
output_model = T.matrix("output_model")

def set_value_at_position(a_location, a_value, output_model):
    zeros = T.zeros_like(output_model)
    zeros_subtensor = zeros[a_location[0], a_location[1]]
    return T.set_subtensor(zeros_subtensor, a_value)

result, updates = theano.scan(fn=set_value_at_position,
                              outputs_info=None,
                              sequences=[location, values],
                              non_sequences=output_model)

assign_values_at_positions = theano.function(inputs=[location, values, output_model], outputs=result)

# test
test_locations = numpy.asarray([[1, 1], [2, 3]], dtype=numpy.int32)
test_values = numpy.asarray([42, 50], dtype=numpy.float32)
test_output_model = numpy.zeros((5, 5), dtype=numpy.float32)
print(assign_values_at_positions(test_locations, test_values, test_output_model))

[[[  0.   0.   0.   0.   0.]
  [  0.  42.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]

 [[  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.  50.   0.]
  [  0.   0.   0.   0.   0.]
  [  0.   0.   0.   0.   0.]]]


## 예제5

In [34]:
import theano
from theano import tensor as T

W = theano.shared('W_values') # we assume that ``W_values`` contains the
                            # initial values of your weight matrix

bvis = theano.shared('bvis_values')
bhid = theano.shared('bhid_values')

trng = T.shared_randomstreams.RandomStreams(1234)

def OneStep(vsample) :
    hmean = T.nnet.sigmoid(theano.dot(vsample, W) + bhid)
    hsample = trng.binomial(size=hmean.shape, n=1, p=hmean)
    vmean = T.nnet.sigmoid(theano.dot(hsample, W.T) + bvis)
    return trng.binomial(size=vsample.shape, n=1, p=vmean,
                         dtype=theano.config.floatX)

sample = theano.tensor.vector()

values, updates = theano.scan(OneStep, outputs_info=sample, n_steps=10)

gibbs10 = theano.function([sample], values[-1], updates=updates)

TypeError: unsupported operand type(s) for +: 'TensorVariable' and 'SharedVariable'

In [24]:
a = theano.shared(1)
values, updates = theano.scan(lambda: {a: a+1}, n_steps=10)

In [25]:
b = a + 1
c = updates[a] + 1
f = theano.function([], [b, c], updates=updates)

print(b)
print(c)
print(a.get_value())

Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
1


In [26]:
W = theano.shared(W_values) # we assume that ``W_values`` contains the
                            # initial values of your weight matrix

bvis = theano.shared(bvis_values)
bhid = theano.shared(bhid_values)

trng = T.shared_randomstreams.RandomStreams(1234)

# OneStep, with explicit use of the shared variables (W, bvis, bhid)
def OneStep(vsample, W, bvis, bhid):
    hmean = T.nnet.sigmoid(theano.dot(vsample, W) + bhid)
    hsample = trng.binomial(size=hmean.shape, n=1, p=hmean)
    vmean = T.nnet.sigmoid(theano.dot(hsample, W.T) + bvis)
    return trng.binomial(size=vsample.shape, n=1, p=vmean,
                     dtype=theano.config.floatX)

sample = theano.tensor.vector()

# The new scan, with the shared variables passed as non_sequences
values, updates = theano.scan(fn=OneStep,
                              outputs_info=sample,
                              non_sequences=[W, bvis, bhid],
                              n_steps=10)

gibbs10 = theano.function([sample], values[-1], updates=updates)

NameError: name 'W_values' is not defined

In [ ]:
# The new scan, using strict=True
values, updates = theano.scan(fn=OneStep,
                              outputs_info=sample,
                              non_sequences=[W, bvis, bhid],
                              n_steps=10,
                              strict=True)

In [27]:
def oneStep(u_tm4, u_t, x_tm3, x_tm1, y_tm1, W, W_in_1, W_in_2,  W_feedback, W_out):

  x_t = T.tanh(theano.dot(x_tm1, W) + \
               theano.dot(u_t,   W_in_1) + \
               theano.dot(u_tm4, W_in_2) + \
               theano.dot(y_tm1, W_feedback))
  y_t = theano.dot(x_tm3, W_out)

  return [x_t, y_t]

In [28]:
W = T.matrix()
W_in_1 = T.matrix()
W_in_2 = T.matrix()
W_feedback = T.matrix()
W_out = T.matrix()

u = T.matrix() # it is a sequence of vectors
x0 = T.matrix() # initial state of x has to be a matrix, since
                # it has to cover x[-3]
y0 = T.vector() # y0 is just a vector since scan has only to provide
                # y[-1]


([x_vals, y_vals], updates) = theano.scan(fn=oneStep,
                                          sequences=dict(input=u, taps=[-4,-0]),
                                          outputs_info=[dict(initial=x0, taps=[-3,-1]), y0],
                                          non_sequences=[W, W_in_1, W_in_2, W_feedback, W_out],
                                          strict=True)
     # for second input y, scan adds -1 in output_taps by default

/Users/kcho/Dropbox/Knowledges/python/virtualEnv/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


## 예제6

In [29]:
def power_of_2(previous_power, max_value):
    return previous_power*2, theano.scan_module.until(previous_power*2 > max_value)

max_value = T.scalar()
values, _ = theano.scan(power_of_2,
                        outputs_info = T.constant(1.),
                        non_sequences = max_value,
                        n_steps = 1024)

f = theano.function([max_value], values)

print f(45)

[  2.   4.   8.  16.  32.  64.]
